In [1]:
import praw
import datetime
import csv
import pickle
import os

In [22]:
visited = set()
if os.path.isfile('visited_comments.txt'):
    with open("visited_comments.txt", 'r') as infile:
        for line in infile:
            visited.add(line.strip())
print(len(visited))

423959


In [21]:
user_agent = "Test/0.1 by maxoboe"
with open('account_info.txt', 'r') as file:
    lines = file.read().splitlines()
    username = lines[0]
    password = lines[1]
    clientID = lines[2]
    secretID = lines[3]
reddit = praw.Reddit(client_id=clientID,
                     client_secret=secretID,
                     user_agent=user_agent)

with open('all_submissions.csv', 'r', encoding='utf-8') as infile, \
    open('all_comments.csv', 'a+', encoding='utf-8', newline='') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ['title', 'selftext', 'ups', 'downs', 'date', 'time', 'comment', 'c_ups', 'c_downs']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
#     writer.writeheader()
    for row in reader:
        if row['comments'] is '0':
            continue
        if row['id'] in visited: continue
        try:
            submission = reddit.submission(id=row['id'])
            submission.comments.replace_more(limit=None)
            for top_level_comment in submission.comments:
                dateTime = datetime.datetime.fromtimestamp(top_level_comment.created)
                writer.writerow({'title':submission.title, 'selftext': row['selftext'], 'ups': row['ups'], 'downs': row['downs'],
                                 'date': dateTime.date(), 'time': dateTime.time(), 'comment': top_level_comment.body, 
                                 'c_ups': top_level_comment.ups, 'c_downs': top_level_comment.downs})
            visited.add(row['id'])
            with open('visited_comments.txt', 'a+') as outfile:
                outfile.write(row['id'] + '\n')
        except:
            print('Execution failed on ' + row['title'])

Execution failed on Near Death Experiences?
Execution failed on What do you think about the parallel universes theory?


In [26]:
import csv 
with open('all_comments.csv', 'r', encoding='utf-8') as infile, \
    open('processed_comments.csv', 'w+', encoding='utf-8', newline='') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ['title', 'selftext', 'ups', 'downs', 'date', 'time', 'comment', 'c_ups', 'c_downs']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in reader:
        if row['comment'] == '[deleted]': continue
        if row['comment'] == '[removed]': continue
        if 'This message was automatic' in row['comment']: continue
        if 'Thank you for your submission! Unfortunately' in row['comment']: continue
        writer.writerow(row)
